In [39]:
import pandas as pd

In [40]:
# Importation des données - on garde uniquement les colonnes et les dates qui nous intéressent

chunk_size = 1000000
file_path = "tr_mutualfunds S12.csv"
columns_to_keep = ['fdate', 'fundname', 'prc']

chunk_iterator = pd.read_csv(file_path, chunksize=chunk_size, usecols=columns_to_keep)

processed_chunks = []

for i, chunk in enumerate(chunk_iterator):
    chunk_filtered = chunk[columns_to_keep]
    chunk_filtered['fdate'] = pd.to_datetime(chunk_filtered['fdate'], errors='coerce')
    chunk_filtered = chunk_filtered[(chunk_filtered['fdate'] >= '1975-01-01') & (chunk_filtered['fdate'] <= '2006-12-31')]

    processed_chunks.append(chunk_filtered)

data = pd.concat(processed_chunks)

In [41]:
data = data.groupby(['fdate', 'fundname']).agg({'prc': 'mean'}).reset_index()

for fonds in data['fundname'].unique():

    group = data[data['fundname'] == fonds]
    group = group.sort_values('fdate')
    
    # On identifie la prochaine date attendue
    group['next_expected_date'] = group['fdate'] + pd.DateOffset(months=3)
    group['next_date'] = group['fdate'].shift(-1)
    valid_rows = (group['next_date'] == group['next_expected_date']) | group['next_date'].isna()
    valid_data = group.loc[valid_rows]

    # Calculer les rendements
    valid_data['prev_prc'] = valid_data['prc'].shift(1)
    valid_data['return'] = (valid_data['prc'] - valid_data['prev_prc']) / valid_data['prev_prc']
    
    return valid_data.dropna(subset=['return'])

In [42]:
# Calcul des rendements pour chaque fonds
data = data.groupby(['fdate', 'fundname']).agg({'prc': 'mean'}).reset_index()
result = data.groupby('fundname').apply(process_fund_group).reset_index(drop=True)

C:\Users\gabri\AppData\Local\Temp\ipykernel_110632\4082492189.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = data.groupby('fundname').apply(process_fund_group).reset_index(drop=True)


In [43]:
# Création du dataframe final
result = result.drop(columns=['next_expected_date', 'next_date', 'prev_prc', 'prc'])

In [44]:
output_file_path = 'mutual_funds_1980_2006_v2.csv'
result.to_csv(output_file_path, index=False)